In [6]:
import spacy
import json
from datetime import datetime, timedelta
import typing as _type
import re
nlp = spacy.load('en_core_web_sm')

quantity_pos = ["NUM"]
quantity_tag = ["CD"]
quantity_identifiers = ["¼", "½", "¾", "⅓", "⅔",
                        "⅕", "⅖", "⅗", "⅘", "⅙", "⅚", "⅛", "⅜", "⅝", "⅞", "\d+\/\d+"]
quantity_unit_identifiers = ["g", "gms", "grams", "l", "litres", "ml", "mL", "L", "dl", "dL", "teaspoon", "tablespoon", "cup", "pinch", "pinche", "piece", "pieces", "pinches", "handful",
                             "pint", "quart", "quarters", "gallon", "mg", "pound", "ounce", "mm", "cm", "mililitres", "kg", "kilograms", "tsp", "tbsp"]


In [7]:
strings = [
    'Whole wheat flour 3/4 cup',

    'Gram flour (besan) 1/4 cup',

    'Salt to taste',

    'Peanuts 1 1/2 teaspoons',

    'Turmeric powder 3/4 teaspoon',

    'Red chilli powder 3/4 teaspoon',

    'Asafoetida 2 pinches',

    'Oil 2 tablespoons',

    'Desi ghee 3 tablespoon',

    'Dried red chillies 2',

    'Fenugreek seeds (methi dana) 1/4 teaspoon',

    'Mustard seeds 3/4 teaspoon',

    'Curry leaves 10',

    'Jaggery (gur), grated 1 teaspoon',

    'Green chilli paste 1 1/2 teaspoon',

    'Onion , finely chopped 1 small',

    'Fresh coriander leaves, chopped 2 tablespoons',
]


phrases = [
    "I am going",
    "200g prawns",
    "200g crab meat",
    "1 large potato, cooked until just soft & grated",
    "4 curry leaves, chopped",
    "handful of fresh coriander leaves",
    "mustard oil for frying",
    "1 tsp turmeric",
    "1 fresh chilli, finely chopped",
    "½ tsp Kashmiri chilli powder",
    "1 tsp garam masala",
    "½ tsp salt or to taste",
    "juice from 1 lemon",
    "3 tbsp gram flour",
    "1 tsp chilli flakes",
]

phrases2=[
"1 tsp cumin seeds",
"1 bunch fresh coriander, leaves and stems",
"½ bunch fresh mint leaves,  ",
"4 garlic cloves",
"3cm piece of fresh ginger",
"4 fresh green chillies",
"1 tsp salt",
"½ lemon, juice only",
"1 tbsp oil",
"3 tbsp thick Greek yogurt",
]

phrases3=[
"1/2 cup Semiya (Vermicelli) , roasted",
"1 small cauliflower",
"2 potatoes, peeled cut into 4cm cubes",
"2 tbsp mustard oil",
"1 tsp mustard seeds",
"1 tsp cumin seeds",
"1 onion, finely chopped",
"2 cloves garlic, finely chopped",
"200g/½ tin tomatoes",
"1 tbsp ginger, grated",
"1 tsp salt",
"1 tsp turmeric",
"1 chilli, finely chopped",
"1 tsp dried fenugreek",
"1 tsp garam masala",
"Handful chopped fresh coriander",
]

In [8]:
def get_quantity_identifier_regexp(quantity_identifiers):
    re_quantity_identifier_str = '\w*[' + \
        ''.join(quantity_identifiers) + ']\w*'
    quantity_identifier_regexp = '('+re_quantity_identifier_str+')'
    return quantity_identifier_regexp


def get_quantity_unit_identifier_regexp(quantity_unit_identifiers):
    re_quantity_unit_identifier_str = '(' + \
        '|'.join(quantity_unit_identifiers) + ')'
    quantity_unit_identifier_regexp = re_quantity_unit_identifier_str
    return quantity_unit_identifier_regexp


In [9]:
for phrase in phrases3:
    quantity_identifier_regexp = get_quantity_identifier_regexp(
        quantity_identifiers)
    quantity_unit_identifier_regexp = get_quantity_unit_identifier_regexp(
        quantity_unit_identifiers)
    name = []
    name_string = ""
    quantity = []
    quantity_unit = []
    ingredient = []
    prep_hint = ""
    phrase = " ".join(re.split('(\d+)',phrase))
    doc = nlp(phrase)
    # print("-"*50)
    # print(phrase)
    deleted_tokens = []
    indices = {c.strip().replace(",", ""): i for i,
               c in enumerate(phrase.split())}
    # print(indices)
    for token in doc:
        # print(token.text," ",token.pos_," ",token.dep_," ",token.tag_," ",token.is_stop)
        if((token.pos_ in quantity_pos) or (token.tag_ in quantity_tag) or (re.fullmatch(quantity_identifier_regexp, token.lemma_))):
            quantity.append(token.text)
            deleted_tokens.append(token.i)
        elif(re.fullmatch(quantity_unit_identifier_regexp, token.lemma_)):
            quantity_unit.append(token.text)
            deleted_tokens.append(token.i)
        elif (((token.tag_ in ['NN']) or (token.dep_ in ['nsubj', 'ROOT'])) and (token.pos_ in ['NOUN', 'PROPN'])):
            name.append(token.text)
            for child in token.children:
                # print(child.text," ",child.pos_," ",child.dep_)
                if((child.dep_ in ['amod', 'compound'])):
                    name.append(child.text)
                    deleted_tokens.append(child.i)
            # name.append(token.text)
            # print(name)
            name = [*set(name)]
            # print("name set: ",name)
            deleted_tokens.append(token.i)
    try:
        name_string = " ".join(sorted(name, key=indices.get))
    except:
        # print("indices exception")
        name_string = " ".join(name)
    for k in quantity_unit + quantity:
        # print k
        name_string = name_string.replace(k, "").strip()
    remaining_phrase = []
    for token in doc:
        if token.i not in deleted_tokens:
            remaining_phrase.append(token.text + " ")
    doc = nlp("".join(remaining_phrase))
    prep_hint = "".join(remaining_phrase)
    # print("Remaining Phrase: {}".format("".join(remaining_phrase)))
    print([name_string, " ".join(quantity),
          " ".join(quantity_unit), prep_hint.strip()])


['Semiya', '1 / 2', 'cup', '( Vermicelli ) , roasted']
['small cauliflower', '1', '', '']
['potatoes peeled cut', '2 4', 'cm', ', into   cubes']
['mustard oil', '2', 'tbsp', '']
['mustard seeds', '1', 'tsp', '']
['cumin seeds', '1', 'tsp', '']
['onion', '1', '', ', finely chopped']
['garlic', '2', '', 'cloves , finely chopped']
['tin tomatoes', '200 g/½', '', '']
['ginger', '1', 'tbsp', ', grated']
['salt', '1', 'tsp', '']
['turmeric', '1', 'tsp', '']
['chilli', '1', '', ', finely chopped']
['dried fenugreek', '1', 'tsp', '']
['garam masala', '1', 'tsp', '']
['fresh coriander', '', 'Handful', 'chopped']


In [10]:
import boto3
s3 = boto3.client('s3')

In [11]:
import uuid
def create_temp_file(size, file_name, file_content):
    random_file_name = ''.join([str(uuid.uuid4().hex[:6]), file_name])
    with open(random_file_name, 'w') as f:
        f.write(str(file_content) * size)
    return random_file_name

In [12]:
first_file_name = create_temp_file(300, 'firstfile.txt', 'f')  

In [13]:
with open(first_file_name, "rb") as f:
    s3.upload_fileobj(f, "recipe-tasveers", first_file_name)

In [14]:
def upload_image_to_s3(filepath,name):
    with open(filepath, "rb") as f:
        s3.upload_fileobj(f, "recipe-tasveers", filepath)

In [15]:
upload_image_to_s3("DSC04075.JPG","profile_pic")

FileNotFoundError: [Errno 2] No such file or directory: 'DSC04075.JPG'

In [ ]:
import time
recipe_name = "crab cakes"
image_name = recipe_name.replace(" ","_") + "_" + time.strftime("%Y%m%d-%H%M%S")
print(image_name)


crab_cakes_20230106-210828


In [ ]:
import requests
recipe_title="something"
image_url="https://i.ytimg.com/vi_webp/NaiN7vqPpW4/sddefault.webp"
r = requests.get(image_url).content
image_name = recipe_title.replace(" ","_") + "_" + time.strftime("%Y%m%d-%H%M%S") + ".jpg"
with open (image_name, 'wb') as f:
    f.write(r)

In [ ]:
import re
youtube_section= """url("https://i.ytimg.com/vi_webp/NaiN7vqPpW4/maxresdefault.webp")"""
print(re.split('[()]',youtube_section)[1])

"https://i.ytimg.com/vi_webp/NaiN7vqPpW4/maxresdefault.webp"


In [ ]:
url = """https://www.youtube.com/embed/NaiN7vqPpW4?rel=0&enablejsapi=1&origin=https%3A%2F%2Fwww.harighotra.co.uk"""
regExp = """/^.*((youtu.be\/)|(v\/)|(\/u\/\w\/)|(embed\/)|(watch\?))\??v?=?([^#&?]*).*/"""
match = re.findall(regExp,url)
print(match)

[]


In [ ]:
from urllib.parse import urlparse, parse_qs


def get_id(url):
    u_pars = urlparse(url)
    quer_v = parse_qs(u_pars.query).get('v')
    if quer_v:
        return quer_v[0]
    pth = u_pars.path.split('/')
    if pth:
        return pth[-1]

In [ ]:
im_url="""https://i.ytimg.com/vi_webp/{}/maxresdefault.webp""".format(get_id(url))
print(im_url)

https://i.ytimg.com/vi_webp/NaiN7vqPpW4/maxresdefault.webp
